In [3]:
import numpy as np
import polars as pl
import pandas as pd
import os
from scripts.group_wise import split_data_by_group_size, load_label_features
import gc


old_model_dir = "model_output/selected_features_xgb/several_group/v1_add_cluster/v1_model"

model_dir = "model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/"
# 先把每個分群都讀一遍feature importance
split_labels = ["small", "medium", "large", 'xlarge']

label_features = load_label_features(
    model_dir=old_model_dir,
    split_labels=split_labels,
    top_n= [100, 131, 133, 139])

# 讀 train data
train_filled = pl.read_parquet("data/train_filled_v1_with_cluster.parquet")

# 呼叫新版本分群切分
result = split_data_by_group_size(
    df=train_filled,
    bins=[0, 212, 517, 1200 , None],
    labels=split_labels,
    label_features=label_features,
    output_dir= os.path.join(model_dir, "train_dataset"),

)

# 清理記憶體
import gc
del train_filled
gc.collect()



✅ small: 取前 100 個特徵

✅ medium: 取前 131 個特徵

✅ large: 取前 133 個特徵

✅ xlarge: 取前 139 個特徵
small: 100 features
medium: 131 features
large: 133 features
xlarge: 139 features
✅ small: 4336349 rows, approx 2597.07 MB
💾 已寫入 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/train_dataset/small.parquet
✅ medium: 4386076 rows, approx 3095.34 MB
💾 已寫入 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/train_dataset/medium.parquet
✅ large: 4689134 rows, approx 3407.59 MB
💾 已寫入 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/train_dataset/large.parquet
✅ xlarge: 4733813 rows, approx 3548.41 MB
💾 已寫入 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/train_dataset/xlarge.parquet
✅ 分群統計：
shape: (4, 4)
┌────────────────┬──────────┬────────────┬────────────────────┐
│ group_category ┆ n_groups ┆ total_rows ┆ avg_rows_per_group │
│ ---            ┆ ---      ┆ ---        ┆ ---                │
│ str            ┆ u32

33

In [ ]:
import os
import gc
import xgboost as xgb
import numpy as np
import polars as pl
from scripts.group_wise import prepare_train_val_split
from scripts.hitrate import compute_hitrate_at_3
import json
# 自訂模型儲存目錄
os.makedirs(model_dir, exist_ok=True)
print(f"✅ Model folder: {model_dir}")


# 你的xgb_params

xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 13,
    'min_child_weight': 1,
    'subsample': 0.9,
    'colsample_bytree': 1,
    'lambda': 1.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 儲存成JSON
params_path = os.path.join(model_dir, "xgb_params.json")

with open(params_path, "w") as f:
    json.dump(xgb_params, f, indent=4)

print(f"✅ 參數已存檔: {params_path}")


all_groups_val_np = []
all_y_val_np = []
all_val_preds = []
hitrate_per_group = {}

for label in split_labels:
    print(f"\n🚀 處理分群: {label}: 有{len(label_features[label])}個features")
    dataset_path = os.path.join(model_dir,"train_dataset")
    data = pl.read_parquet(os.path.join(dataset_path, f"{label}.parquet"))

    split_result = prepare_train_val_split(
        result={"split_data": {label: data}},
        split_label=label,
        feature_cols=label_features[label],
        train_fraction=0.8
    )
    xgb_model = xgb.train(
        xgb_params,
        split_result["dtrain"],
        num_boost_round=500,
        evals=[(split_result["dtrain"], "train"), (split_result["dval"], "val")],
        early_stopping_rounds=50,
        verbose_eval=20
    )

    model_path = os.path.join(model_dir, f"xgb_ranker_{label}.bin")
    xgb_model.save_model(model_path)
    print(f"✅ 已儲存模型: {model_path}")

    val_preds = xgb_model.predict(split_result["dval"])

    hitrate = compute_hitrate_at_3(
        split_result["groups_val_np"],
        split_result["y_val_np"],
        val_preds
    )
    hitrate_per_group[label] = hitrate

    all_groups_val_np.append(split_result["groups_val_np"])
    all_y_val_np.append(split_result["y_val_np"])
    all_val_preds.append(val_preds)

    del xgb_model, val_preds, split_result, data
    gc.collect()
    print("🧹 記憶體已清理")

print("\n⚡ 合併所有驗證結果計算 HitRate ⚡")
all_groups_val_np = np.concatenate(all_groups_val_np)
all_y_val_np = np.concatenate(all_y_val_np)
all_val_preds = np.concatenate(all_val_preds)

overall_hitrate = compute_hitrate_at_3(
    all_groups_val_np,
    all_y_val_np,
    all_val_preds
)

print("\n💡 每組 Hitrate：")
for label, h in hitrate_per_group.items():
    print(f"{label}: {h:.4f}")

print(f"\n🎯 全部合併 HitRate: {overall_hitrate:.4f}")

# 將 hitrate 寫入 CSV
hitrate_records = [
    {"split_label": label, "hitrate": h}
    for label, h in hitrate_per_group.items()
]
hitrate_records.append({"split_label": "overall", "hitrate": overall_hitrate})

hitrate_df = pl.DataFrame(hitrate_records)
csv_path = os.path.join(model_dir, "hitrate_summary.csv")
hitrate_df.write_csv(csv_path)
print(f"\n✅ 已儲存所有 Hitrate 結果至 {csv_path}")


✅ Model folder: model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/
✅ 參數已存檔: model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/xgb_params.json

🚀 處理分群: small: 有100個features
✅ Train: 3463517 rows, 67380 groups
✅ Val: 872832 rows, 16846 groups
[0]	train-ndcg@3:0.46888	val-ndcg@3:0.43869
[20]	train-ndcg@3:0.69833	val-ndcg@3:0.54618
[40]	train-ndcg@3:0.75214	val-ndcg@3:0.56179
[60]	train-ndcg@3:0.78876	val-ndcg@3:0.57275
[80]	train-ndcg@3:0.81211	val-ndcg@3:0.58110
[100]	train-ndcg@3:0.83511	val-ndcg@3:0.58711
[120]	train-ndcg@3:0.85157	val-ndcg@3:0.59315
[140]	train-ndcg@3:0.86656	val-ndcg@3:0.59901
[160]	train-ndcg@3:0.87597	val-ndcg@3:0.60247
[180]	train-ndcg@3:0.88770	val-ndcg@3:0.60643
[200]	train-ndcg@3:0.90097	val-ndcg@3:0.61199
[220]	train-ndcg@3:0.91507	val-ndcg@3:0.61563
[240]	train-ndcg@3:0.92767	val-ndcg@3:0.62015
[260]	train-ndcg@3:0.94015	val-ndcg@3:0.62552
[280]	train-ndcg@3:0.94889	val-ndcg@3:0.62954
[300]	train-ndcg@3:0.95637	va

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_84149/1418141467.py:66: UserWarning: [05:23:01] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1748292887431/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_path)


✅ 已儲存模型: model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/xgb_ranker_small.bin
✅ HitRate@3 (groups size in [10, inf]): 0.6903
🧹 記憶體已清理

🚀 處理分群: xlarge: 有139個features
✅ Train: 3806845 rows, 1322 groups
✅ Val: 926968 rows, 331 groups
[0]	train-ndcg@3:0.03914	val-ndcg@3:0.00946
[20]	train-ndcg@3:0.70774	val-ndcg@3:0.11200
[40]	train-ndcg@3:0.84225	val-ndcg@3:0.13426
[60]	train-ndcg@3:0.90083	val-ndcg@3:0.15994
[80]	train-ndcg@3:0.92373	val-ndcg@3:0.17663
[100]	train-ndcg@3:0.93895	val-ndcg@3:0.17656
[120]	train-ndcg@3:0.95114	val-ndcg@3:0.18522
[140]	train-ndcg@3:0.95921	val-ndcg@3:0.19127
[160]	train-ndcg@3:0.96440	val-ndcg@3:0.18602
[180]	train-ndcg@3:0.97225	val-ndcg@3:0.17886
[200]	train-ndcg@3:0.97951	val-ndcg@3:0.19238
[220]	train-ndcg@3:0.98535	val-ndcg@3:0.19619
[240]	train-ndcg@3:0.98834	val-ndcg@3:0.20263
[260]	train-ndcg@3:0.99059	val-ndcg@3:0.20979
[280]	train-ndcg@3:0.99264	val-ndcg@3:0.21090
[300]	train-ndcg@3:0.99320	val-ndcg@3:0.21353
[320]	train-nd

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_84149/1418141467.py:66: UserWarning: [05:26:59] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1748292887431/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_path)


✅ HitRate@3 (groups size in [10, inf]): 0.2326
🧹 記憶體已清理

⚡ 合併所有驗證結果計算 HitRate ⚡
✅ HitRate@3 (groups size in [10, inf]): 0.6797

💡 每組 Hitrate：
small: 0.6903
xlarge: 0.2326

🎯 全部合併 HitRate: 0.6797

✅ 已儲存所有 Hitrate 結果至 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/hitrate_summary.csv


In [5]:
# 指定你的模型資料夾
from scripts.group_wise import export_xgb_feature_importance, export_common_feature_ranks

# 逐一跑
for lbl in split_labels:
    print(f"\n🚀 處理模型: {lbl}")
    df_importance = export_xgb_feature_importance(
        model_dir=model_dir,
        label=lbl,
    )

df_common = export_common_feature_ranks(
    labels=split_labels,
    model_importance_dir=f"{model_dir}/model_importance")



🚀 處理模型: small
✅ 已讀取模型 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/xgb_ranker_small.bin
✅ 已輸出所有特徵重要性到 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/model_importance/feature_importance_small_all_features.csv

🚀 處理模型: xlarge
✅ 已讀取模型 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/xgb_ranker_xlarge.bin
✅ 已輸出所有特徵重要性到 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/model_importance/feature_importance_xlarge_all_features.csv
✅ 已讀 small: 100 rows
✅ 已讀 xlarge: 139 rows

🎯 所有模型共同出現特徵: 89

✅ 已輸出共同特徵到 model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model//model_importance/common_features_with_ranks.csv


# Prediction

In [6]:
import json
import os
from scripts.group_wise import split_data_by_group_size_test, load_used_features_from_importance_csv
import numpy as np
import polars as pl

# 讀取
test_filled = pl.read_parquet("data/test_filled_v1_with_cluster.parquet")

model_dir = "model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/"

# === 讀取 grouping_config.json ===
grouping_config_path = os.path.join(model_dir, "train_dataset", "grouping_config.json")
with open(grouping_config_path, "r") as f:
    grouping_config = json.load(f)

# 取出 bins 和 labels
bins_from_config = grouping_config["bins"]
labels_from_config = grouping_config["labels"]

# 把第一個bin改成0
bins_fixed = bins_from_config.copy()
bins_fixed[0] = 0

print(f"✅ Loaded bins: {bins_fixed}")
print(f"✅ Loaded labels: {labels_from_config}")

# 排除欄
exclude_cols = [
    'Id', 'ranker_id', 'selected', 'profileId', 'requestDate', 'frequentFlyer',
    'legs0_departureAt', 'legs0_arrivalAt', 'legs1_departureAt', 'legs1_arrivalAt',
    'miniRules0_percentage', 'miniRules1_percentage',
    'pricingInfo_passengerCount'
]

# 讀特徵
feature_dict = load_used_features_from_importance_csv(
    model_importance_dir=os.path.join(model_dir, "model_importance"),
    labels=labels_from_config
)

# 看一個範例
print(feature_dict[labels_from_config[1]])

# 執行分群
result = split_data_by_group_size_test(
    test_filled,
    bins=bins_fixed,
    labels=labels_from_config
)


✅ Loaded bins: [0, 212, 517, 1200, 8237]
✅ Loaded labels: ['small', 'medium', 'large', 'xlarge']
✅ small: 共 100 個用到的特徵
✅ medium: 共 131 個用到的特徵
✅ large: 共 133 個用到的特徵
✅ xlarge: 共 139 個用到的特徵
['total_num_transfers_rank', 'legs0_segments0_flightNumber', 'legs0_mean_cabin', 'price_per_duration_rank', 'both_legs_carrier_all_same', 'total_weighted_mean_cabin', 'price_percentile', 'legs1_mean_cabin', 'price_from_median_zscore', 'total_duration', 'price_per_duration', 'pricingInfo_isAccessTP', 'baggage_total_rank', 'legs0_arrivalAt_hour', 'legs0_segments0_baggageAllowance_quantity', 'price_minus_fee_rank', 'legs1_segments2_cabinClass', 'price_per_fee_rank', 'legs0_departureAt_hour', 'legs1_segments0_operatingCarrier_code', 'legs0_segments1_key_view_count', 'totalPrice_rank', 'leg0_flight_view_count', 'days_before_departure', 'price_per_fee', 'legs1_main_carrier', 'legs0_segments0_cabinClass', 'price_per_tax', 'legs0_segments0_operatingCarrier_code', 'isVip', 'legs1_departureAt_hour', 'legs0_segme

In [7]:
import xgboost as xgb
import numpy as np
from scripts.group_wise import prepare_prediction
import os


all_groups_val_np = []
all_X_val_np = []
all_val_preds = []
all_rows_val_np = []

for label in labels_from_config:
    print(f"\n🚀 處理分群: {label}")

    # 先讀模型
    model_path = os.path.join(model_dir, f"xgb_ranker_{label}.bin")
    xgb_model = xgb.Booster(model_file=model_path)

    # 提取模型feature順序
    model_features = xgb_model.feature_names
    if model_features is None:
        raise ValueError(f"模型 {model_path} 沒有feature_names，請確認訓練時有指定 feature_names")

    print(f"✅ 模型 {label} feature順序共 {len(model_features)} 欄")

    # 準備資料
    split_result = prepare_prediction(
        result=result,
        split_label=label,
        feature_cols=model_features,  # 直接用模型順序
    )

    # 再次驗證 feature 完整性
    feature_in_data = set(split_result["feature_cols"])
    missing_in_data = [f for f in model_features if f not in feature_in_data]
    if missing_in_data:
        raise ValueError(f"❌ 模型feature在資料中找不到: {missing_in_data}")

    # 把X_np依照模型特徵順序重新選取
    df_for_reorder = result["split_data"][label].select(model_features)
    X_np_sorted = df_for_reorder.to_numpy()

    dval = xgb.DMatrix(X_np_sorted, feature_names=model_features)
    dval.set_group(split_result["group_sizes"])

    # 預測
    val_preds = xgb_model.predict(dval)
    print(val_preds)
    all_groups_val_np.append(split_result["groups_np"])
    all_val_preds.append(val_preds)
    all_rows_val_np.append(split_result["global_row_np"])


import gc
del result, dval, val_preds, xgb_model, X_np_sorted, df_for_reorder
gc.collect()
# 合併
all_groups_val_np = np.concatenate(all_groups_val_np)
all_val_preds = np.concatenate(all_val_preds)
all_rows_val_np = np.concatenate(all_rows_val_np)

# 合併回test_filled
preds_df = pl.DataFrame({
    "global_row_nr": all_rows_val_np,
    "selected": all_val_preds
})

test_filled_with_preds = (
    test_filled
    .with_row_count("global_row_nr")
    .join(preds_df, on="global_row_nr", how="left")
    .drop("global_row_nr")
)

restored_clean = test_filled_with_preds.drop("selected")
equal = test_filled.equals(restored_clean)
print("✅ 是否與原始 test_filled 完全一致：", equal)
import gc
del test_filled, restored_clean
gc.collect()
# 合併


🚀 處理分群: small
✅ 模型 small feature順序共 100 欄
✅ small 分組大小: 38714
[-1.8716532  -0.47046304 -3.0198894  ... -2.859554   -0.22575012
 -3.5759726 ]

🚀 處理分群: medium
✅ 模型 medium feature順序共 131 欄
✅ medium 分組大小: 4355
[-3.694815  -1.3953366 -5.044826  ... -5.0210505 -6.381163  -6.5140624]

🚀 處理分群: large
✅ 模型 large feature順序共 133 欄
✅ large 分組大小: 1310
[-2.9849567 -3.6664867 -5.2830358 ... -7.190782  -6.0931416 -5.876604 ]

🚀 處理分群: xlarge
✅ 模型 xlarge feature順序共 139 欄
✅ xlarge 分組大小: 852
[-4.128732  -4.3085356 -3.0356688 ... -6.049929  -6.221242  -8.049209 ]


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_84149/818430053.py:70: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("global_row_nr")


✅ 是否與原始 test_filled 完全一致： True


0

In [8]:
from scripts.group_wise import export_submission_parquets
export_submission_parquets(
    test_filled_with_preds=test_filled_with_preds,   # 你的帶有 selected 分數的 DataFrame
    output_dir=model_dir
)


✅ 已儲存原始 submission: model_output/selected_features_xgb/several_group/v1_add_cluster/v2_model/raw_submission.parquet
shape: (6_897_776, 4)
┌──────────┬─────────────────────────────────┬───────────┬───────────────────┐
│ Id       ┆ ranker_id                       ┆ selected  ┆ __index_level_0__ │
│ ---      ┆ ---                             ┆ ---       ┆ ---               │
│ i64      ┆ str                             ┆ f64       ┆ i64               │
╞══════════╪═════════════════════════════════╪═══════════╪═══════════════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -3.694815 ┆ 18144679          │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -1.395337 ┆ 18144680          │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -5.044826 ┆ 18144681          │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -3.270484 ┆ 18144682          │
│ 18144683 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -1.641779 ┆ 18144683          │
│ …        ┆ …                               ┆ …         ┆ …            